In [1]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests as req
from tqdm import tqdm
import os

In [2]:
# 제목
title_list = []
# 날짜
date_list = []
# 첨부 파일
f_name_list = [] # 파일명
link_list = [] # url

In [3]:
# HTML 컨텐츠 수집
for i in tqdm(range(1, 54)) : 
    min_url = f"https://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761&pageIndex={i}"
    res = req.get(min_url)
    soup = bs(res.content, "html.parser")
    
    links = soup.select("div.col.m2.s2.x3.fileLink a")
    titles = soup.select("span.titlesub")
    dates = soup.select("span.date")

    for i in range(len(titles)) :
        title_list.append(titles[i].text)
    for i in range(len(dates)) :
        date_list.append(dates[i].text)

    cnt = 0 # 중요! : i번 째 페이지에서 컨텐츠 갯수만큼 돌고, 다시 초기화하여 다음 페이지로 넘어가기 위함
    for i in links :
        filename = i.get_text().strip()
        
        # 확장자가 hwp인 파일만 수집
        if filename[-3:] == 'hwp':
            # 파일명
            filename = f"minutes_{dates[cnt].text}.hwp"
            f_name_list.append(filename)
            cnt += 1
            
            # 파일의 주소 값
            file_addr = "http://bok.or.kr" + i["href"]
            link_list.append(file_addr)

print(len(title_list), len(date_list), len(f_name_list), len(link_list))

100%|██████████| 53/53 [01:06<00:00,  1.26s/it]

526 526 393 393


In [4]:
# 한국은행 금융통화위원회 의사록 페이지 HTML구조가 바뀌기 전까지로 list 길이 수정
title_list = title_list[:300]
date_list = date_list[:300]
f_name_list = f_name_list[:300]
link_list = link_list[:300]

In [5]:
# 첨부파일 다운로드
for i in tqdm(range(len(link_list))) :
    try :
        res = req.get(link_list[i])
        # dir 생성 or 확인
        if not os.path.isdir("data") :
            os.mkdir("data")
        if res.status_code == 200 :
            file_path = os.path.join("data", f_name_list[i])

            # 파일 열기, 쓰기
            open(file_path, "wb").write(res.content)
    except Exception as e :
        print("Error! : ", e)
        continue
print("Complete😇")

100%|██████████| 300/300 [00:48<00:00,  6.21it/s]

Complete😇


In [6]:
# DataFrame 생성
pd.set_option("display.max_rows", None)
df = pd.DataFrame({
    "title" : title_list,
    "date" : date_list,
    "file" : f_name_list,
    "url" : link_list
})

# 날짜 컬럼 문자열 자르기
df["date"] = df["date"].str.replace("등록일", "")
df

,title,date,file,url
0,금융통화위원회 의사록(2024년도 제10차)(2024.5.23),2024.06.11,minutes_등록일2024.06.11.hwp,http://bok.or.kr/portal/cmmn/file/fileDown.do?...
1,금융통화위원회 의사록(2024년도 제9차)(2024.5.9),2024.05.28,minutes_등록일2024.05.28.hwp,http://bok.or.kr/portal/cmmn/file/fileDown.do?...
2,금융통화위원회 의사록(2024년도 제7차)(2024.4.12),2024.04.30,minutes_등록일2024.04.30.hwp,http://bok.or.kr/portal/cmmn/file/fileDown.do?...
3,금융통화위원회 의사록(2024년도 제6차)(2024.3.28),2024.04.16,minutes_등록일2024.04.16.hwp,http://bok.or.kr/portal/cmmn/file/fileDown.do?...
4,금융통화위원회 의사록(2024년도 제5차)(2024.3.14),2024.04.02,minutes_등록일2024.04.02.hwp,http://bok.or.kr/portal/cmmn/file/fileDown.do?...
5,금융통화위원회 의사록(2024년도 제4차)(2024.2.22),2024.03.12,minutes_등록일2024.03.12.hwp,http://bok.or.kr/portal/cmmn/file/fileDown.do?...
6,금융통화위원회 의사록(2024년도 제3차)(2024.2.8),2024.02.27,minutes_등록일2024.02.27.hwp,http://bok.or.kr/portal/cmmn/file/fileDown.do?...
7,금융통화위원회 의사록(2024년도 제2차)(2024.1.25),2024.02.13,minutes_등록일2024.02.13.hwp,http://bok.or.kr/portal/cmmn/file/fileDown.do?...
8,금융통화위원회 의사록(2024년도 제1차)(2024.1.11),2024.01.30,minutes_등록일2024.01.30.hwp,http://bok.or.kr/portal/cmmn/file/fileDown.do?...
9,금융통화위원회 의사록(2023년도 제24차)(2023.12.28),2024.01.16,minutes_등록일2024.01.16.hwp,http://bok.or.kr/portal/cmmn/file/fileDown.do?...


In [7]:
df.to_csv("data/minutes_info.csv", encoding = "utf8")